In [1]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True,
)

mp3_file_path = "sample.wav"


result = pipe(mp3_file_path)
print(result["text"])


/home/ubuntu/miniconda3/envs/model-compression/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cuda:0
/home/ubuntu/miniconda3/envs/model-compression/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Whisper did not predict an ending timestamp, which can happen if audio is cut of

 Возможно ли, что супергерои появятся в реальной жизни? Мог бы это и быть? Супергерои? Что имеется ввиду под супергероями? Это же генетическая тема, ну то есть... Типа то, что скрафтят усиленных людей? Ну конечно это возможно. Ну типа да, это возможно. Грубо говоря, усиленные люди это будут те, у кого скорее всего будет возможность купить это. Ну прям как в кино супергерои, ну блин, прям такого не будет конечно. Ну я думаю, в будущем могут развить там какую-то у кого-то усиленную регенерацию Или усиленную скорость, силу, блять Так даже сейчас же, ну вот есть те же самые астероиды и прочее Ты видел, как некоторые люди выглядят? То есть они выглядят нереалистично абсолютно То есть такого представить нельзя, такого быть в принципе не может То, как некоторые выглядят То есть если посмотреть, там какие-то фото атлетов, допустим, лет 50 или 100 назад То есть, ну, обманывают природу Прямо сейчас уже Субтитры сделал DimaTorzok


In [2]:
def get_model_size(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()
    size_all_mb = (param_size + buffer_size) / 1024**2
    print(f"Размер модели (вычислено): {size_all_mb:.2f} MB")
    return size_all_mb


In [3]:
get_model_size(model=model)

Размер модели (вычислено): 2943.97 MB


2943.974609375

In [4]:
import torch
import torchaudio
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor # Изменено AutoModel
import time
import psutil
from datasets import load_dataset, Audio
import evaluate
import os
import gc
from tqdm.auto import tqdm

# --- Конфигурация ---
MODEL_ID = "openai/whisper-large-v3" # <--- ИЗМЕНЕНО: ID вашей новой модели
DATASET_ID = "mozilla-foundation/common_voice_16_1" # Датасет для оценки
DATASET_NAME = "ru" # Используем русскую часть Common Voice
DATASET_SPLIT = "test"
NUM_SAMPLES_FOR_QUALITY_TEST = 100 # Количество сэмплов для быстрой оценки, можно увеличить
TARGET_SAMPLE_RATE = 16000 # Whisper ожидает 16kHz

# --- Определение устройства и типа данных ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Whisper v3 хорошо работает с float16 на GPU для ускорения и экономии памяти
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
print(f"Используемое устройство: {device}, тип данных: {torch_dtype}")

# --- 1. Загрузка модели и процессора ---
print(f"Загрузка процессора для {MODEL_ID}...")
# AutoProcessor для Whisper загрузит и feature_extractor, и tokenizer
processor = AutoProcessor.from_pretrained(MODEL_ID)

print(f"Загрузка модели {MODEL_ID}...")
# Используем AutoModelForSpeechSeq2Seq для моделей Whisper
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    MODEL_ID,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True, # Полезно для больших моделей
    use_safetensors=True   # Более безопасный и часто быстрый формат загрузки
)
model.to(device) # Перемещаем модель на выбранное устройство
model.eval() # Переводим модель в режим оценки
print("Модель и процессор загружены.")

# --- 2. Загрузка метрик WER и CER ---
wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")
print("Метрики WER и CER загружены.")

# --- 3. Загрузка и подготовка датасета ---
print(f"Загрузка датасета {DATASET_ID}, конфигурация: {DATASET_NAME}, сплит: {DATASET_SPLIT}...")
# Загружаем указанное количество сэмплов
dataset = load_dataset(DATASET_ID, DATASET_NAME, split=f"{DATASET_SPLIT}[:{NUM_SAMPLES_FOR_QUALITY_TEST}]", trust_remote_code=True)
print(f"Загружено {len(dataset)} сэмплов для оценки.")
# Приводим аудио в датасете к нужной частоте дискретизации (16kHz для Whisper)
dataset = dataset.cast_column("audio", Audio(sampling_rate=TARGET_SAMPLE_RATE))
print(f"Аудио в датасете преобразовано к {TARGET_SAMPLE_RATE} Hz.")

# --- 4. Функция для инференса и декодирования (АДАПТИРОВАНА ДЛЯ WHISPER) ---
def transcribe_audio_whisper(batch_audio_sample):
    raw_audio = batch_audio_sample["array"]
    sampling_rate = batch_audio_sample["sampling_rate"] # Должно быть TARGET_SAMPLE_RATE

    if len(raw_audio) == 0:
        return ""

    # Процессор Whisper готовит input_features из сырого аудио
    # Передаем сырой аудио массив и его частоту дискретизации
    input_features = processor(raw_audio, sampling_rate=sampling_rate, return_tensors="pt").input_features
    
    # Перемещаем input_features на то же устройство и с тем же dtype, что и модель
    input_features = input_features.to(device, dtype=torch_dtype)

    # Для многоязычных моделей Whisper важно указать язык для декодирования,
    # чтобы получить наилучшее качество на конкретном языке.
    # `processor.get_decoder_prompt_ids` помогает сформировать нужные токены для `generate`.
    forced_decoder_ids = processor.get_decoder_prompt_ids(language="russian", task="transcribe")

    with torch.no_grad():
        # Генерируем последовательность токенов
        predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
    
    # Декодируем ID токенов в текст.
    # `skip_special_tokens=True` убирает специальные токены (например, токены языка, тишины).
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    
    return transcription.strip()

# --- 5. Оценка качества на датасете ---
predictions = []
references = []

print(f"\nНачало оценки качества на {NUM_SAMPLES_FOR_QUALITY_TEST} сэмплах для {MODEL_ID}...")
for i in tqdm(range(len(dataset)), desc="Оценка качества"):
    sample = dataset[i]
    reference_text = sample["sentence"]

    # Пропускаем сэмплы с пустыми эталонными транскрипциями
    if not reference_text or reference_text.strip() == "":
        continue

    try:
        # Используем функцию, адаптированную для Whisper
        predicted_text = transcribe_audio_whisper(sample["audio"])
        
        # Приводим к нижнему регистру для консистентного сравнения
        predictions.append(predicted_text.lower())
        references.append(reference_text.lower())

    except Exception as e:
        audio_path_info = sample.get('path', f'индекс {i}') # Пытаемся получить путь, если есть
        print(f"Ошибка при обработке сэмпла {audio_path_info}: {e}")
        # В случае ошибки можно добавить пустую строку или специальный маркер,
        # но лучше пропустить, чтобы не искажать сильно метрику, если ошибок мало.
        # Для данного скрипта, если ошибка, пара не добавится, и количество обработанных будет меньше.

    # Периодическая очистка памяти GPU, если используется CUDA
    if device.type == 'cuda':
        torch.cuda.empty_cache()
    gc.collect()

# --- 6. Расчет и вывод метрик WER и CER ---
if predictions and references:
    # Убедимся, что количество предсказаний и эталонов совпадает
    # (на случай, если какие-то сэмплы были пропущены из-за ошибок или пустых эталонов)
    num_evaluated_samples = len(predictions)
    print(f"\nРасчет метрик для {num_evaluated_samples} успешно обработанных сэмплов.")

    if num_evaluated_samples > 0:
        wer_score = wer_metric.compute(predictions=predictions, references=references)
        cer_score = cer_metric.compute(predictions=predictions, references=references)
        print(f"Качество модели (Word Error Rate - WER) для {MODEL_ID}: {wer_score:.4f}")
        print(f"Качество модели (Character Error Rate - CER) для {MODEL_ID}: {cer_score:.4f}")
    else:
        print("Не было собрано ни одного валидного предсказания для расчета метрик.")
else:
    print("\nНе удалось собрать предсказания и/или эталоны для расчета WER/CER.")

# Вывод нескольких примеров для визуальной оценки
print("\nПримеры транскрипций:")
for i in range(min(5, len(predictions))):
    print(f"Эталон     : {references[i]}")
    print(f"Предсказание: {predictions[i]}\n")

Используемое устройство: cuda, тип данных: torch.float16
Загрузка процессора для openai/whisper-large-v3...
Загрузка модели openai/whisper-large-v3...
Модель и процессор загружены.
Метрики WER и CER загружены.
Загрузка датасета mozilla-foundation/common_voice_16_1, конфигурация: ru, сплит: test...
Загружено 100 сэмплов для оценки.
Аудио в датасете преобразовано к 16000 Hz.

Начало оценки качества на 100 сэмплах для openai/whisper-large-v3...


Оценка качества: 100%|██████████| 100/100 [00:59<00:00,  1.69it/s]


Расчет метрик для 100 успешно обработанных сэмплов.
Качество модели (Word Error Rate - WER) для openai/whisper-large-v3: 0.1714
Качество модели (Character Error Rate - CER) для openai/whisper-large-v3: 0.0713

Примеры транскрипций:
Эталон     : масштабы финансово-экономического кризиса и темпы его распространения застали самых опытных специалистов мира врасплох.
Предсказание: масштабы финансово-экономического кризиса и темпового распространения застали самых опытных специалистов мира врасплох.

Эталон     : к сожалению, эти предложения не нашли отражения в тексте.
Предсказание: к сожалению, эти предложения не носили отражения в тексте.

Эталон     : мы настоятельно призываем посла танина незамедлительно провести неофициальное пленарное заседание для обсуждения реформы совета.
Предсказание: мы настоятельно призываем посла тонино незамедлительно провести неофициальное пленарное заседание для обсуждения реформы совета.

Эталон     : толпа озвереет, будет тереться, ощетинит ножки стоглава

In [5]:
import torch
import torchaudio
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor # Изменено AutoModel
import time
import psutil
import os
import gc
from datasets import load_dataset, Audio
from tqdm.auto import tqdm # Добавим для наглядности подготовки сэмплов

# --- Конфигурация ---
MODEL_ID = "openai/whisper-large-v3" # <--- ИЗМЕНЕНО: ID вашей новой модели
DATASET_ID = "mozilla-foundation/common_voice_16_1"
DATASET_NAME = "ru"
DATASET_SPLIT = "test"
NUM_SAMPLES_FOR_TIME_TEST = 20
TARGET_SAMPLE_RATE = 16000 # Whisper ожидает 16kHz
NUM_WARMUP_RUNS = 5

# --- Определение устройства и типа данных ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32 # float16 для GPU
print(f"Используемое устройство: {device}, тип данных: {torch_dtype}")

# --- 1. Загрузка модели и процессора ---
print(f"Загрузка процессора для {MODEL_ID}...")
processor = AutoProcessor.from_pretrained(MODEL_ID)

print(f"Загрузка модели {MODEL_ID}...")
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    MODEL_ID,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
    use_safetensors=True
)
model.to(device) # Сразу перемещаем на основное устройство (GPU или CPU)
model.eval()
print("Модель и процессор загружены.")

# Подготовка forced_decoder_ids для русского языка (для GPU и CPU инференса)
# Это нужно делать один раз, так как они не зависят от конкретного аудиосэмпла
forced_decoder_ids = processor.get_decoder_prompt_ids(language="russian", task="transcribe")

# --- 2. Подготовка сэмплов ---
# Сэмплы будут готовиться как сырые аудио массивы,
# так как Whisper процессор принимает их напрямую.
# Предобработка в input_features будет происходить непосредственно перед инференсом.
print(f"Загрузка {NUM_SAMPLES_FOR_TIME_TEST + NUM_WARMUP_RUNS} аудио сэмплов для замеров...")
dataset_for_timing = load_dataset(
    DATASET_ID,
    DATASET_NAME,
    split=f"{DATASET_SPLIT}[:{NUM_SAMPLES_FOR_TIME_TEST + NUM_WARMUP_RUNS}]",
    trust_remote_code=True
)
dataset_for_timing = dataset_for_timing.cast_column("audio", Audio(sampling_rate=TARGET_SAMPLE_RATE))

# Сохраняем сырые аудиоданные
raw_audio_list = []
print(f"Подготовка аудиоданных...")
for i in tqdm(range(NUM_SAMPLES_FOR_TIME_TEST + NUM_WARMUP_RUNS), desc="Подготовка аудио"):
    raw_audio_list.append(dataset_for_timing[i]["audio"]["array"])
print("Аудио сэмплы подготовлены.")


# --- Переменные для результатов ---
time_gpu_ms_avg = "N/A"
vram_usage_mb = "N/A"

# --- 3. Измерение на GPU (если доступно) ---
if device.type == 'cuda': # Проверяем основное устройство
    print("\n--- Измерение на GPU ---")
    # Модель уже на GPU и в нужном dtype
    
    print(f"Прогрев GPU ({NUM_WARMUP_RUNS} запусков)...")
    for i in range(NUM_WARMUP_RUNS):
        raw_audio = raw_audio_list[i]
        # Предобработка процессором и перемещение на GPU
        input_features = processor(raw_audio, sampling_rate=TARGET_SAMPLE_RATE, return_tensors="pt").input_features.to(device, dtype=torch_dtype)
        with torch.no_grad():
            _ = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
            
    torch.cuda.synchronize(device) # Убедимся, что прогрев завершен
    torch.cuda.reset_peak_memory_stats(device) # Сбрасываем счетчик пиковой памяти ПОСЛЕ прогрева
    
    gpu_times = []
    print(f"Замеры времени инференса GPU ({NUM_SAMPLES_FOR_TIME_TEST} запусков)...")
    
    # Первый реальный замер для VRAM
    raw_audio_for_vram = raw_audio_list[NUM_WARMUP_RUNS]
    input_features_for_vram = processor(raw_audio_for_vram, sampling_rate=TARGET_SAMPLE_RATE, return_tensors="pt").input_features.to(device, dtype=torch_dtype)
    with torch.no_grad():
        _ = model.generate(input_features_for_vram, forced_decoder_ids=forced_decoder_ids)
    torch.cuda.synchronize(device)
    vram_usage_mb = torch.cuda.max_memory_allocated(device) / (1024 * 1024)
    del input_features_for_vram # Освобождаем память
    if NUM_SAMPLES_FOR_TIME_TEST == 0:
         time_gpu_ms_avg = 0 # или "N/A"

    for i in range(NUM_WARMUP_RUNS, NUM_WARMUP_RUNS + NUM_SAMPLES_FOR_TIME_TEST):
        raw_audio = raw_audio_list[i]
        input_features = processor(raw_audio, sampling_rate=TARGET_SAMPLE_RATE, return_tensors="pt").input_features.to(device, dtype=torch_dtype)
        
        torch.cuda.synchronize(device)
        start_time = time.perf_counter()
        with torch.no_grad():
            _ = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
        torch.cuda.synchronize(device)
        end_time = time.perf_counter()
        gpu_times.append((end_time - start_time) * 1000)
        del input_features # Освобождаем память после каждого инференса
        
    if gpu_times:
        time_gpu_ms_avg = sum(gpu_times) / len(gpu_times)
    
    print(f"Среднее время инференса на GPU: {time_gpu_ms_avg if isinstance(time_gpu_ms_avg, str) else f'{time_gpu_ms_avg:.2f}'} ms")
    print(f"Пиковое использование VRAM (измерено на первом реальном запуске): {vram_usage_mb if isinstance(vram_usage_mb, str) else f'{vram_usage_mb:.2f}'} MB")
    
    # Модель остается на GPU для возможного CPU-тестирования, если torch_dtype был изменен для CPU
    # Но если CPU-тест идет после GPU-теста, модель нужно переместить на CPU
    # gc.collect() и torch.cuda.empty_cache() полезны
    gc.collect()
    torch.cuda.empty_cache()
else:
    print("CUDA (GPU) не доступна. Пропуск измерений на GPU.")

# --- 4. Измерение на CPU ---
print("\n--- Измерение на CPU ---")
# Перемещаем модель на CPU и устанавливаем dtype=float32, если она была на GPU с float16
# Если изначально device был 'cpu', то модель уже там и в float32 (или как загрузилась)
cpu_device = torch.device("cpu")
model.to(cpu_device) # Гарантированно перемещаем на CPU
if torch_dtype == torch.float16 and device.type == "cuda": # Если модель была на GPU в float16
    model = model.to(torch.float32) # Переводим в float32 для CPU, т.к. CPU float16 медленный
    print("Модель переведена в float32 для CPU-тестирования.")
current_cpu_dtype = model.dtype # Узнаем текущий dtype модели на CPU
print(f"Модель на CPU с dtype: {current_cpu_dtype}")


process = psutil.Process(os.getpid())
ram_before_cpu_warmup_mb = process.memory_info().rss / (1024 * 1024)

print(f"Прогрев CPU ({NUM_WARMUP_RUNS} запусков)...")
for i in range(NUM_WARMUP_RUNS):
    raw_audio = raw_audio_list[i]
    # Предобработка процессором (результат на CPU) и приведение к dtype модели
    input_features = processor(raw_audio, sampling_rate=TARGET_SAMPLE_RATE, return_tensors="pt").input_features.to(dtype=current_cpu_dtype)
    with torch.no_grad():
        _ = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)

cpu_times = []
print(f"Замеры времени инференса CPU ({NUM_SAMPLES_FOR_TIME_TEST} запусков)...")
ram_after_cpu_warmup_mb = process.memory_info().rss / (1024 * 1024)

for i in range(NUM_WARMUP_RUNS, NUM_WARMUP_RUNS + NUM_SAMPLES_FOR_TIME_TEST):
    raw_audio = raw_audio_list[i]
    input_features = processor(raw_audio, sampling_rate=TARGET_SAMPLE_RATE, return_tensors="pt").input_features.to(dtype=current_cpu_dtype)
    start_time = time.perf_counter()
    with torch.no_grad():
        _ = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
    end_time = time.perf_counter()
    cpu_times.append((end_time - start_time) * 1000)
    del input_features # Освобождаем память

if cpu_times:
    time_cpu_ms_avg = sum(cpu_times) / len(cpu_times)
else:
    time_cpu_ms_avg = "N/A"

ram_usage_mb = process.memory_info().rss / (1024 * 1024)

print(f"Среднее время инференса на CPU: {time_cpu_ms_avg if isinstance(time_cpu_ms_avg, str) else f'{time_cpu_ms_avg:.2f}'} ms")
print(f"Использование RAM (после инференсов на CPU): {ram_usage_mb:.2f} MB")
print(f"(RAM до прогрева CPU: {ram_before_cpu_warmup_mb:.2f} MB, после прогрева CPU: {ram_after_cpu_warmup_mb:.2f} MB)")


# --- 5. Итоговые результаты для таблицы ---
print("\n--- Итоговые результаты для таблицы ---")
print(f"Модель: {MODEL_ID}")
print(f"Время инференса (CPU, ms): {time_cpu_ms_avg if isinstance(time_cpu_ms_avg, str) else f'{time_cpu_ms_avg:.2f}'}")
print(f"Время инференса (GPU, ms): {time_gpu_ms_avg if isinstance(time_gpu_ms_avg, str) else f'{time_gpu_ms_avg:.2f}'}")
print(f"Использование RAM (MB): {ram_usage_mb:.2f}")
print(f"Использование VRAM (MB): {vram_usage_mb if isinstance(vram_usage_mb, str) else f'{vram_usage_mb:.2f}'}")

Используемое устройство: cuda, тип данных: torch.float16
Загрузка процессора для openai/whisper-large-v3...
Загрузка модели openai/whisper-large-v3...
Модель и процессор загружены.
Загрузка 25 аудио сэмплов для замеров...
Подготовка аудиоданных...


Подготовка аудио: 100%|██████████| 25/25 [00:00<00:00, 161.62it/s]

Аудио сэмплы подготовлены.

--- Измерение на GPU ---
Прогрев GPU (5 запусков)...


Замеры времени инференса GPU (20 запусков)...
Среднее время инференса на GPU: 383.90 ms
Пиковое использование VRAM (измерено на первом реальном запуске): 6146.68 MB

--- Измерение на CPU ---
Модель переведена в float32 для CPU-тестирования.
Модель на CPU с dtype: torch.float32
Прогрев CPU (5 запусков)...
Замеры времени инференса CPU (20 запусков)...
Среднее время инференса на CPU: 4504.25 ms
Использование RAM (после инференсов на CPU): 7772.48 MB
(RAM до прогрева CPU: 7679.34 MB, после прогрева CPU: 7772.51 MB)

--- Итоговые результаты для таблицы ---
Модель: openai/whisper-large-v3
Время инференса (CPU, ms): 4504.25
Время инференса (GPU, ms): 383.90
Использование RAM (MB): 7772.48
Использование VRAM (MB): 6146.68
